In [1]:
from datetime import datetime, timezone
import pandas as pd
from textblob import TextBlob
import textstat


In [3]:
df = pd.read_csv("US_youtube_trending_data.csv")
#df = pd.read_csv("CA_youtube_trending_data.csv")
#df = pd.read_csv("GB_youtube_trending_data.csv")

# Read US, CA, and GB
# Build a model to classify really popular from not so popular videos among the trending dataset
# Find a threshold based on views

In [35]:
# US, CA, and GB
# Build a model to classify really popular from not so popular videos among the trending dataset
# Find a threshold based on views
"""
# Below code shows how new columns were made:
df['titleLen'] = df['title'].str.len()
df['descriptionLen'] = df['description'].str.len()
df['UserActivityRate'] = ((df['likes'] + df['comment_count']) / df['view_count']) * 100
df['UserActivityRate'] = df['UserActivityRate'].round(2)
def counttags(taglist):
    if taglist == '[None]':
        return 0
    return len(taglist.split('|'))

df['numTags'] = df['tags'].apply(counttags)

# Finding Top 10% for different metrics:

lowviews = df['view_count'].quantile(0.9)
df['isTopTenViews'] = (df['view_count'] >= lowviews).astype(int)

lowlikes = df['likes'].quantile(0.9)
df['isTopTenLikes'] = (df['likes'] >= lowlikes).astype(int)

lowcomment = df['comment_count'].quantile(0.9)
df['isTopTenComments'] = (df['comment_count'] >= lowcomment).astype(int)

lowactive = df['UserActivityRate'].quantile(0.9)
df['isTopTenActivity'] = (df['UserActivityRate'] >= lowactive).astype(int)

df['likeToViewRatio'] = df['likes'] / df['view_count']
df['likeToViewRatio'] = df['likeToViewRatio'].round(2)

lowratio = df['likeToViewRatio'].quantile(0.9)
df['isTopTenLikeViewRatio'] = (df['likeToViewRatio'] >= lowratio).astype(int)
"""

"\n# Commented out to show how new columns were made\ndf['titleLen'] = df['title'].str.len()\ndf['descriptionLen'] = df['description'].str.len()\ndf['UserActivityRate'] = ((df['likes'] + df['comment_count']) / df['view_count']) * 100\ndf['UserActivityRate'] = df['UserActivityRate'].round(2)\ndef counttags(taglist):\n    if taglist == '[None]':\n        return 0\n    return len(taglist.split('|'))\n\ndf['numTags'] = df['tags'].apply(counttags)\n\nlowviews = df['view_count'].quantile(0.9)\ndf['isTopTenViews'] = (df['view_count'] >= lowviews).astype(int)\n\nlowlikes = df['likes'].quantile(0.9)\ndf['isTopTenLikes'] = (df['likes'] >= lowlikes).astype(int)\n\nlowcomment = df['comment_count'].quantile(0.9)\ndf['isTopTenComments'] = (df['comment_count'] >= lowcomment).astype(int)\n\nlowactive = df['UserActivityRate'].quantile(0.9)\ndf['isTopTenActivity'] = (df['UserActivityRate'] >= lowactive).astype(int)\n\ndf['likeToViewRatio'] = df['likes'] / df['view_count']\ndf['likeToViewRatio'] = df['li

In [41]:
"""
categoryDict = {1: "Film & Animation",
2: "Autos & Vehicles",
10: "Music",
15: "Pets & Animals",
17: "Sports",
18: "Short Movies",
19: "Travel & Events",
20: "Gaming",
21: "Videoblogging",
22: "People & Blogs",
23: "Comedy",
24: "Entertainment",
25: "News & Politics",
26: "Howto & Style",
27: "Education",
28: "Science & Technology",
29: "Nonprofits & Activism",
30: "Movies",
31: "Anime/Animation",
32: "Action/Adventure",
33: "Classics",
34: "Comedy",
35: "Documentary",
36: "Drama",
37: "Family",
38: "Foreign",
39: "Horror",
40: "Sci-Fi/Fantasy",
41: "Thriller",
42: "Shorts",
43: "Shows",
44: "Trailers"
}
df['CategoryName'] = df['categoryId'].map(categoryDict)
"""

In [48]:
#df.to_csv('NEW_youtube_data_US.csv', index=False)


In [39]:
df.shape

(268787, 46)

In [30]:
"""
df['title'] = df['title'].astype(str)
df['description'] = df['description'].astype(str)

def sentiment(text):
    return round(TextBlob(text).sentiment.polarity,2)

    
# Sentiment is a value from -1 to 1 which represents most negative to most positive
df['titleSentiment'] = df['title'].apply(sentiment)
df['descriptionSentiment'] = df['description'].apply(sentiment)

def rc(text):

    return textstat.text_standard(text, float_output=True)

# Returns the estimated school grade level required to understand the text
df['titleReadabilityConsensus'] = df['title'].apply(rc)
df['descriptionReadabilityConsensus'] = df['description'].apply(rc)



# Measures readability (grade level required to understand the text) for foreign learners of English
df['titleForeignReadability'] = df['title'].apply(textstat.mcalpine_eflaw)
df['descriptionForeignReadability'] = df['description'].apply(textstat.mcalpine_eflaw)

def ttr(text):
    return textstat.reading_time(text, ms_per_char=14.69)
# Time to read in seconds, ms_per_char is recommended by the library
df['titleTimeToRead'] = df['title'].apply(ttr)
df['descriptionTimeToRead'] = df['description'].apply(ttr)

df['descriptionSyllableCount'] = df['description'].apply(textstat.syllable_count)
df['titleSyllableCount'] = df['title'].apply(textstat.syllable_count)

def wc(text):
    return textstat.lexicon_count(text, removepunct=True)
df['descriptionWordCount'] = df['description'].apply(wc)
df['titleWordCount'] = df['title'].apply(wc)

def avgsent(tags):
    if tags == '[None]':
        return None
    taglist = tags.split('|')
    numtags = len(taglist)
    avg = 0
    for tag in taglist:
        avg += TextBlob(tag).sentiment.polarity
    avg = avg / numtags
    return round(avg, 2)

df['avgTagSentiment'] = df['tags'].apply(avgsent)

def avgrc(tags):
    if tags == '[None]':
        return None
    taglist = tags.split('|')
    numtags = len(taglist)
    avg = 0
    for tag in taglist:
        avg += textstat.text_standard(tag, float_output=True)
    avg = avg / numtags
    return round(avg, 2)
df['avgTagReadabilityConsensus'] = df['tags'].apply(avgrc)

def avgFR(tags):
    if tags == '[None]':
        return None
    taglist = tags.split('|')
    numtags = len(taglist)
    avg = 0
    for tag in taglist:
        avg += textstat.mcalpine_eflaw(tag)
    avg = avg / numtags
    return round(avg, 2)
df['avgTagForeignReadability'] = df['tags'].apply(avgFR)

def avgsyl(tags):
    if tags == '[None]':
        return None
    taglist = tags.split('|')
    numtags = len(taglist)
    avg = 0
    for tag in taglist:
        avg += textstat.syllable_count(tag)
    avg = avg / numtags
    return round(avg, 2)
df['avgTagSyllableCount'] = df['tags'].apply(avgsyl)
"""




In [35]:
"""
def parsedt(publishedAt):
    dt = datetime.fromisoformat(publishedAt.replace("Z", "+00:00"))
    midnight = datetime.combine(dt.date(), datetime.min.time(), tzinfo=timezone.utc)
    SecondsFromMidnight = abs((dt - midnight).total_seconds())
    HoursFromMidnight = round(SecondsFromMidnight / 3600, 2)
    return HoursFromMidnight
df['HoursFromMidnight'] = df['publishedAt'].apply(parsedt)

def getweekday(pubAt):
    pubAt = pubAt[0:10]
    dt = datetime.fromisoformat(pubAt)
    return dt.strftime('%A')

df['PublishWeekday'] = df['publishedAt'].apply(getweekday)

def getmonth(pubAt):
    pubAt = pubAt[0:10]
    dt = datetime.fromisoformat(pubAt)
    return dt.strftime('%B')
df['PublishMonth'] = df['publishedAt'].apply(getmonth)
"""


In [47]:
print("Memory usage in bytes:")
print(df.memory_usage(deep=True).sum())

Memory usage in bytes:
952609953


In [ ]:
df.head(5)

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,...,titleSyllableCount,descriptionWordCount,titleWordCount,HoursFromMidnight,PublishWeekday,avgTagSentiment,avgTagReadabilityConsensus,avgTagForeignReadability,avgTagSyllableCount,PublishMonth
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,...,8,43,7,19.34,Tuesday,0.02,1.07,2.13,2.13,August
1,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11T17:00:10Z,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12T00:00:00Z,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,...,13,99,8,17.00,Tuesday,-0.01,4.60,3.08,4.32,August
2,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11T16:34:06Z,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12T00:00:00Z,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353787,...,12,27,11,16.57,Tuesday,0.06,7.10,2.00,3.33,August
3,kXLn3HkpjaA,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11T16:38:55Z,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12T00:00:00Z,xxl freshman|xxl freshmen|2020 xxl freshman|20...,496771,23251,...,13,88,7,16.65,Tuesday,-0.01,1.00,2.30,2.70,August
4,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11T15:10:05Z,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12T00:00:00Z,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,45802,...,14,206,9,15.17,Tuesday,0.03,3.85,2.18,3.12,August
